# Tight End Data Cleaning

In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import re
import sys
sys.path.append('../src/')
from helpers import *

In [2]:
# Import all data
# assign path
path, dirs, files = next(os.walk("../data/raw_data/te_stats/"))
file_count = len(files)
# create empty list
dataframes_list = []
 
# append datasets to the list
for i in range(file_count):
    temp_df = pd.read_csv("../data/raw_data/te_stats/"+files[i], thousands=',')
    dataframes_list.append(temp_df)
    
# Array to store dataframe names
dataframes_names = []

for i in range(len(dataframes_list)):
    # If a player value is null, drop the row
    dataframes_list[i] = dataframes_list[i].dropna(subset=['Player'])
   
   # Rename all the dataframes as name of original csv file with date at the end (i.e. 2018_qb_stats.csv --> qb_stats_2018)
    globals()[files[i][5:].removesuffix('.csv')+'_'+files[i][:4]] = dataframes_list[i]
    dataframes_names.append(files[i][5:].removesuffix('.csv')+'_'+files[i][:4])

In [3]:
#Convert Percentage String to Percentage Float
# float(x.strip('%'))/100
def percent (row):
    if row['% TM'] == None:
        return 0
    return float(row['% TM'].strip('%'))/100

def merge(df1, df2):
        # # merge stats and advanced stats
        df3= pd.merge(left=df1, right=df2, on='Player', how='left').fillna(0)
        
        drop_y(df3)
        rename_x(df3)
        
        # Fix Names
        df3.insert(2, 'NAME', df3.apply (lambda row: team_remove(row), axis=1))
        df3 = df3.drop(columns='Player')
        
        # Adjust Target Share
        df3.insert(2, 'TGT_SHARE', df3.apply (lambda row: percent(row), axis=1))
        df3 = df3.drop(columns='% TM')
    
        
        # Convert all objects to floats
        cols = df3.columns
        df3[cols[3:]] = df3[cols[3:]].apply(pd.to_numeric, errors ='coerce')
        
        return df3

In [4]:
for i in range(len(dataframes_names)-1):
    if dataframes_names[i][-4:] == dataframes_names[i+1][-4:]:
        df1 = globals()[dataframes_names[i]] 
        df2 = globals()[dataframes_names[i+1]]
        
        # rename columns in stats dataframe
        df1 = df1.rename(
            columns={'Rank':'RANK', 'YDS':'REC_YDS', 'RZ TGT': 'RZ_TGT'}).drop(
                columns=['10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'CATCHABLE', 'DROP', 'LNG', 'YBC', 'YBC/R'])
        df2 = df2.rename(
            columns={'Rank':'RANK', 'YDS':'REC_YDS', 'TD':'REC_TD', 'ATT':'RUS_ATT', 'YDS.1':'RUS_YDS', 'TD.1':'RUS_TD'}).drop(
                columns=['ROST','20+', 'LG'])
            
        # Return new dataframe
        globals()[dataframes_names[i+1]] = merge(df1, df2)
        # 
# Change dataframes_names to contian only the mutated dataframes
dataframes_names = dataframes_names[1::2]

In [5]:
te_stats_2022

,RANK,NAME,TGT_SHARE,G,REC,REC_YDS,Y/R,AIR,AIR/R,YAC,...,BRKTKL,TGT,RZ_TGT,REC_TD,RUS_ATT,RUS_YDS,RUS_TD,FL,FPTS,FPTS/G
0,1.0,Travis Kelce,0.248,17.0,110.0,1338.0,12.2,690.0,6.3,648.0,...,13.0,152.0,30.0,12.0,2.0,5.0,0.0,1.0,316.3,18.6
1,2.0,George Kittle,0.171,15.0,60.0,765.0,12.8,383.0,6.4,382.0,...,4.0,86.0,15.0,11.0,0.0,0.0,0.0,1.0,200.5,13.4
2,3.0,Taysom Hill,0.026,16.0,9.0,77.0,8.6,40.0,4.4,37.0,...,0.0,13.0,3.0,2.0,96.0,575.0,7.0,0.0,145.8,9.1
3,4.0,T.J. Hockenson,0.229,17.0,86.0,914.0,10.6,518.0,6.0,396.0,...,7.0,129.0,18.0,6.0,0.0,0.0,0.0,1.0,215.4,12.7
4,5.0,Mark Andrews,0.243,15.0,73.0,847.0,11.6,604.0,8.3,243.0,...,1.0,113.0,17.0,5.0,3.0,8.0,0.0,0.0,190.5,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,102.0,Maxx Williams,0.005,3.0,3.0,18.0,6.0,12.0,4.0,6.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.8,1.6
102,103.0,Stone Smartt,0.007,3.0,4.0,17.0,4.3,10.0,2.5,7.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.7,1.9
103,104.0,Stephen Anderson,0.011,9.0,3.0,9.0,3.0,3.0,1.0,6.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.9,0.4
104,105.0,Devin Asiasi,0.007,5.0,2.0,5.0,2.5,5.0,2.5,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.5


In [6]:
# export new csv files
for df in dataframes_names:
    globals()[df].to_csv(f'../data/clean_data/te_stats_clean/{df}_clean.csv', index=False)